In [4]:
import warnings
warnings.filterwarnings("ignore")

import findspark
findspark.init()

from pyspark.sql.window import Window
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

import pyarrow.parquet as pq
import tensorflow as tf
import pandas as pd
import numpy as np
import re

In [2]:
parrallelism = 8

In [3]:
spark = SparkSession.builder \
                    .appName('BERT App') \
                    .config("spark.dynamicAllocation.enabled", False) \
                    .config("spark.driver.memory", "4g") \
                    .config("spark.cores.max", parrallelism) \
                    .config("spark.executor.instances", parrallelism) \
                    .config("spark.executor.cores", 1) \
                    .config("spark.executor.memory", "4g") \
                    .enableHiveSupport() \
                    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [6]:
N = 300

goods = spark.table("ruten.goods").alias("g")
orders = spark.table("ruten.orders").alias("o")

ranked_category = (
    orders
    .filter( ( trim(col("category_name")) != "" ) & ( trim(col("category_name")) != '其他' ) )
    .groupBy(trim(col("category_name")).alias("category_name"))
    .agg(count(col("item_id")).alias("item_count"))
    .withColumn("rank", rank().over(Window.orderBy(col("item_count").desc())))
)

items = (
    orders
    .join(ranked_category.alias("c"), on="category_name", how="inner")
    .where(col("c.rank") <= N)
    .join(goods, on="item_id", how="inner")
    .select(col("c.category_name"), col("o.item_id"), col("o.item_name"), col("g.seller_nickname"))
    .withColumn("item_name", lower(trim(regexp_replace(regexp_replace(col("item_name"), '&#\w+;', ''), '[^\u4e00-\u9fffa-zA-Z0-9]+', ' '))) )
    .where(col("item_name") != '')
    .distinct()
)

all_items_df = items.toPandas()
all_items_df

,category_name,item_id,item_name,seller_nickname
0,其他汽車百貨,10060831298854,纖維彈簧透氣墊 沙發 汽車 電腦椅 輪椅 透氣椅墊 透氣坐墊 透氣座墊 小寵物墊 通風 散熱...,blackjack88
1,EPSON,10061017636459,墨水王 評價9000 lxmark epson canon hp 高品質台灣填充墨水,tim0813x
2,遊戲王,10061027590698,歡樂屋 遊戲王亞洲英文版第一版套牌 遊戲sdy 海馬sdk 絕版商品 無底價,chen133
3,角色扮演,11061115445908,md 石中劍 有盒書,zakkwang
4,其他漫畫,11061124384090,夢想王國 二手漫畫 少年老千 1 3完 山田秋太郎 東立 無釘有章,wucalvinwu
...,...,...,...,...
586929,其他汽車精品,22152612754311,愛淨小舖 福士打蠟機藍色底盤,xoxo0717
586930,其他機車零組件,22152618990735,jf asdf66999訂購明細,jf-moto
586931,其他漫畫,22152623216709,2本合售 漫畫書 無章釘 小叮噹彩色長篇 宇宙開拓史 上 下 藤子 f 不二雄 青文 脫頁無...,alixson8
586932,其他家庭雜貨,22152624223809,泡澡桶大人 可折疊浴桶 家用全身成人浴缸汗蒸兒童沐浴盆洗澡盆神器,biboyo


In [7]:
# Undersampling
yugioh_item_df = all_items_df.query("category_name == '遊戲王'").sample(n=40062, random_state=123)

remaining_item_df = all_items_df.query("category_name not in ['其他', '遊戲王'] ")

In [8]:
items_df = pd.concat([yugioh_item_df, remaining_item_df])
items_df

,category_name,item_id,item_name,seller_nickname
112686,遊戲王,22052435638325,遊戲王 ps 16 舞蹈士兵 普卡r112 98新 會換新卡套,lieko05122
327189,遊戲王,22040537909416,app store 收藏出清 韓紙 igas jp020 閃刀姬 羅潔 半鑽,benson082012
254820,遊戲王,22042685668201,遊戲王 1101 rotd jp062 三戰之才 金亮,f7211062002
50888,遊戲王,22115788204448,g r 拍賣 1105 dama jp036 魔鍵召獸 古獸希亞拉波斯 普卡 90 95分,william5554123
138280,遊戲王,22139340277011,馬亞卡鋪 日紙 wpp2 jp006 神速召喚 普卡,a61004575
...,...,...,...,...
586928,臂章/識別章,22152609716797,鐵鳥迷 現貨超商 空軍f 16v viper軟膠pvc機種章 有魔鬼氈,maxwombat
586929,其他汽車精品,22152612754311,愛淨小舖 福士打蠟機藍色底盤,xoxo0717
586930,其他機車零組件,22152618990735,jf asdf66999訂購明細,jf-moto
586931,其他漫畫,22152623216709,2本合售 漫畫書 無章釘 小叮噹彩色長篇 宇宙開拓史 上 下 藤子 f 不二雄 青文 脫頁無...,alixson8


In [ ]:
spark.createDataFrame(items_df).repartition(4) \
        .write.mode("overwrite") \
        .saveAsTable("ruten.items")

In [ ]:
items_df.to_parquet("/tmp/items.parquet", engine='pyarrow', compression='snappy')

In [ ]:
!hdfs dfs -copyFromLocal /tmp/items.parquet /ruten/items.parquet